# Challenge 1 - T-test

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_re`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](https://researchbasics.education.uconn.edu/t-test/) and [this Quora](https://www.quora.com/What-is-the-difference-between-a-paired-and-unpaired-t-test). Make sure you understand when to use which type of t-test. 

In [1]:
# Import libraries

import pandas as pd
from statsmodels.stats.weightstats import ztest as ztest
from scipy.stats import ttest_ind, norm, t, ttest_rel
from scipy.stats import f_oneway

#### Import dataset

In this challenge we will work on the Pokemon dataset you have used last week. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.).

In [2]:
# Import dataset

pokemon = pd.read_csv('Pokemon.csv')

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [30]:
def t_test_features(s1, s2, alternative="two-sided", 
                    features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    
    results = {stat : ttest_ind(s1[stat], s2[stat], alternative = alternative)[1] for stat in features}
    
    return results

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [31]:
# Subset
legendary = pokemon[pokemon['Legendary'] == True]
non_legendary = pokemon[pokemon['Legendary'] == False]

# Calculating p-values
results = t_test_features(legendary, non_legendary, alternative="greater")
results

{'HP': 1.6653238424230954e-15,
 'Attack': 3.913626501602666e-24,
 'Defense': 7.921113047213627e-13,
 'Sp. Atk': 3.157457885213633e-41,
 'Sp. Def': 9.219904790204797e-27,
 'Speed': 1.1770377218449218e-21,
 'Total': 1.5476228734826412e-52}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [32]:
# H0: stat.mean.legendary == stat.mean.non_legendary
# H1: stat.mean.legendary > stat.mean.non_legendary

# Given that p-value < 0.05 for each stat, we reject H0 for each stat. We have enough evidence to accept H1.
# Thus, legendary pokemon are better in all stats.

In [46]:
legendary['Total'].mean()

637.3846153846154

In [47]:
non_legendary['Total'].mean()

417.21360544217686

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [33]:
# Subset
gen_1 = pokemon[pokemon['Generation'] == 1]
gen_2 = pokemon[pokemon['Generation'] == 2]

# Calculating p-values
results = t_test_features(gen_1, gen_2)
results

{'HP': 0.13791881412813622,
 'Attack': 0.24050968418101457,
 'Defense': 0.5407630349194362,
 'Sp. Atk': 0.14119788176331508,
 'Sp. Def': 0.16781226231606386,
 'Speed': 0.0028356954812578704,
 'Total': 0.5599140649014442}

#### What conclusions can you make?

In [34]:
# H0: stat.mean.gen_1 == stat.mean.gen_2
# H1: stat.mean.gen_1 != stat.mean.gen_2

# Given that p-value > 0.05 for all stats except Speed, we cannot reject H0 (except for Speed)
# There's no significant difference in stats between Gen_1 and Gen_2, except for the Speed,
# for which there's significant difference between generations.

In [45]:
gen_1['Total'].mean()

426.81325301204816

In [44]:
gen_2['Total'].mean()

418.2830188679245

#### Compare pokemons who have single type vs those having two types.

In [35]:
# Subset
single_types = pokemon[(pokemon['Type 1'].notna()) & (pokemon['Type 2'].isna())]
two_types = pokemon[(pokemon['Type 1'].notna()) & (pokemon['Type 2'].notna())]

# Calculating p-values
results = t_test_features(two_types, single_types, alternative = "greater")
results

{'HP': 0.05530321572215921,
 'Attack': 7.870697833082198e-05,
 'Defense': 1.625297102878502e-08,
 'Sp. Atk': 7.274587020175735e-05,
 'Sp. Def': 5.446652397767198e-05,
 'Speed': 0.012025705397018732,
 'Total': 5.874517504414376e-08}

#### What conclusions can you make?

In [36]:
# H0: stat.mean.two_types == stat.mean.single_types
# H1: stat.mean.two_types > stat.mean.single_types

# For all stats except HP, p-value < 0.05. Therefore, for these stats we have enough evidence
# to reject H0. Thus, pokemon with two types are overall better than those with one type.
# This is not the case for the HP stat, for which we cannot reject H0.

In [40]:
single_types['Defense'].mean()

67.58549222797927

In [41]:
two_types['Defense'].mean()

79.67632850241546

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [37]:
ttest_rel(pokemon['Attack'], pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

In [38]:
ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

#### What conclusions can you make?

In [48]:
# Attack vs Defense
# H0: means are equal
# H1: means are not equal
# Given that p-value is < 0.05, we can reject H0 and accept H1, for which attack and defense
# have significantly different means.

In [49]:
pokemon['Attack'].mean()

79.00125

In [50]:
pokemon['Defense'].mean()

73.8425

In [51]:
# Sp. Atk vs Sp. Def
# H0: means are equal
# H1: means are not equal
# Given that p-value is > 0.05, we do not have enough evidence to reject H0.
# Therefore, there are no significant differences between the means of these stats

In [52]:
pokemon['Sp. Atk'].mean()

72.82

In [53]:
pokemon['Sp. Def'].mean()

71.9025